## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import sys
import os
import pandas as pd
import requests
import gmaps

sys.path.insert(0, "/Users/jackiecho/Desktop/Analysis_Projects/World_Weather_Analysis/")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,40,23.02,light intensity shower rain
1,1,Nikolskoye,RU,59.7035,30.7861,30.70,96,35,2.95,scattered clouds
2,2,Loa Janan,ID,-0.5830,117.0950,70.61,94,98,1.61,overcast clouds
3,3,Westport,US,41.1415,-73.3579,44.91,92,100,5.75,overcast clouds
4,4,Nanlong,CN,31.3533,106.0631,58.53,77,100,4.72,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Loa Janan,ID,-0.5830,117.0950,70.61,94,98,1.61,overcast clouds
6,6,Camocim,BR,-2.9022,-40.8411,79.05,88,86,9.48,heavy intensity rain
9,9,Mar Del Plata,AR,-38.0023,-57.5575,75.22,62,40,21.85,scattered clouds
10,10,Carnarvon,AU,-24.8667,113.6333,77.07,83,40,12.66,scattered clouds
13,13,Avarua,CK,-21.2078,-159.7750,77.05,94,20,2.30,few clouds
14,14,Berbera,SO,10.4396,45.0143,84.13,59,5,2.44,clear sky
15,15,Kavieng,PG,-2.5744,150.7967,80.69,78,100,4.63,light rain
16,16,Hilo,US,19.7297,-155.0900,78.53,76,100,6.91,heavy intensity rain
20,20,Atuona,PF,-9.8000,-139.0333,79.95,72,83,21.14,broken clouds
21,21,Tabou,CI,4.4230,-7.3528,80.47,82,100,10.38,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Loa Janan,ID,70.61,overcast clouds,-0.5830,117.0950,
6,Camocim,BR,79.05,heavy intensity rain,-2.9022,-40.8411,
9,Mar Del Plata,AR,75.22,scattered clouds,-38.0023,-57.5575,
10,Carnarvon,AU,77.07,scattered clouds,-24.8667,113.6333,
13,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
...,...,...,...,...,...,...,...
698,Bowen,AU,77.97,scattered clouds,-20.0167,148.2333,
700,Bilma,NE,87.26,clear sky,18.6853,12.9164,
701,Lakatoro,VU,77.00,overcast clouds,-16.0999,167.4164,
706,Najran,SA,72.25,clear sky,17.4924,44.1277,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
print(len(hotel_df.loc[hotel_df["Hotel Name"] == ""]))

clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

278
15


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Loa Janan,ID,70.61,overcast clouds,-0.5830,117.0950,Kost bu jono
6,Camocim,BR,79.05,heavy intensity rain,-2.9022,-40.8411,Ilha Praia Hotel
9,Mar Del Plata,AR,75.22,scattered clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
10,Carnarvon,AU,77.07,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
13,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
689,Pak Phanang,TH,80.08,moderate rain,8.3511,100.2020,Green Park Inn Hotel
698,Bowen,AU,77.97,scattered clouds,-20.0167,148.2333,Castle Motor Lodge Motel
700,Bilma,NE,87.26,clear sky,18.6853,12.9164,Expeditions Ténére voyages
701,Lakatoro,VU,77.00,overcast clouds,-16.0999,167.4164,Lakatoro Palm Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))